# What we mapped

With this notebook, and with [QGIS TimeManager plug-in](https://plugins.qgis.org/plugins/timemanager/), you will be able to create animated gif and movie to illustrate the work accomplished during a Mapathon event.

Example 
---------------
Idjwi, Democratic Republic of the Congo.

![alt tag](https://github.com/tgrippa/Mapathon_HOT_OSM_WhatWeMapped/blob/master/Example/Mapathon2016accomplishment.gif?raw=true)

# Configuration

## Import libraries

In [170]:
## Import libraries needed for setting parameters of operating system 
import os
import sys

## Import library for temporary files creation 
import tempfile 

## Import Numpy library
import numpy as np

## Import Psycopg2 library (interection with postgres database)
import psycopg2 as pg

## Import Pandas library (View and manipulaiton of tables)
import pandas as pd

## User inputs

In [171]:
## Define a empty dictionnary for saving user inputs
user={}

In [218]:
## Enter the name of the new postgresqgl database
user["dbname"] = "Mapathon"
## Enter the postgresqgl username
user["user"] = "tais"
## Enter postgresqgl Password
user["password"] = "tais"
## Enter postgresqgl host
user["host"] = "localhost"
## Enter DB port
user["port"] = "5432"
## Enter postgresqgl schema
user["schema"] = "belgianmapath2016"

Here after:
- Enter the path to the main directory of Postgresql.

In [173]:
## Enter the path to the osm2pgsql folder
user["pgsqlfolder"]="C:\\Program Files\\PostgreSQL\\9.6\\bin"
## Add environment variables for Postgresql
os.environ['PATH'] = user["pgsqlfolder"] + os.pathsep + os.environ['PATH']

Here after:
- Enter the path to the directory where osm2pgsql in located. 
The command-line based program "osm2pgsql" have to be installed on your computer. See [the official wiki page](http://wiki.openstreetmap.org/wiki/Osm2pgsql) for more information. For Windows user, you have to [dowload the osm2pgsql executable](https://ci.appveyor.com/project/openstreetmap/osm2pgsql/history). Go to the page and find the latest 'artifact'.  




In [174]:
## Enter the path to the osm2pgsql folder
user["osm2pgsqlfolder"]="C:\Users\Admin_ULB\osm2pgsql-bin"
## Add environment variables for osm2pgsql
os.environ['PATH'] = user["osm2pgsqlfolder"] + os.pathsep + os.environ['PATH']

In [175]:
# Define the path to the osm2pgsql default.style file
user["stylefile"]=os.path.join(user["osm2pgsqlfolder"],"default_TG.style")

Here after:
- Enter the path to the folder where the .osm file covering your area of interest will be downloaded. 
OSM data downloading is automated in this script. In case you would manage yourself the retrieving of OSM data, please read the [official wiki page](http://wiki.openstreetmap.org/wiki/Downloading_data) for that purpose. In that case some parts of this notebook should be adapted.

In [176]:
## Enter the path to the folder where the .osm file will be downloaded
user["osmfolder"]="F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016"
## Enter the prefix you want to be used in the PostGIS DB
user["prefixosm"]="osm"

Here after:
- Enter the path to a .shp file (a single polygon feature) corresponding to the area of interest (AOI) which has been mapped. Please, be sure the projection is EPSG 3857.

In [177]:
## Enter the path to the .shp file corresponding to the area of interested which has been mapped
user["aoi_shp"]="F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\AOI_idjwi_2016.shp"

Here after:
- Enter the path to .shp files where the resulting osm layer will be saved.

In [178]:
## Enter the path to the .shp file where the resulting osm layer will be saved - Path for LINES
user["output_lineshp"]="F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\output_line.shp"

In [179]:
## Enter the path to the .shp file where the resulting osm layer will be saved - Path for POLYGONS
user["output_polygonshp"]="F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\output_polygon.shp"

Here after:
- Postgresql tables' name are saved as variables. You could change it only if you want.

In [180]:
## Save postgresqgl table's mane
user["aoi"] = "aoi"
user["bbox"] = "bbox"
user["bbox_coord"] = "bbox_coord"

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Import data in Postgis

## Create postgis extension

Postgis extension will be created if it doesn't exists!

In [183]:
# Connect to the database
db=None
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
cur=db.cursor()
# Execute the query
cur.execute('CREATE EXTENSION IF NOT EXISTS postgis')
# Make the changes to the database persistent
db.commit()
# Close connection with database
cur.close()
db.close()

## Import AOI shapefile

The following part use the "shp2pgsql" program which should normally already be installed since postgis extension have been created in postgresql. See [this quick guide](http://www.bostongis.com/pgsql2shp_shp2pgsql_quickguide.bqg) for more information. 

Fist, a bash file is created according to user parameters. Then, the bash.dat file is pass in an embedded command prompt.

In [184]:
## Build the shp2pgsql command-line
cmdline="set PGPASSWORD="+user["password"]+"\n"
cmdline+="shp2pgsql -s 3857 -d -I"+" "
cmdline+=user["aoi_shp"]+" "+user["schema"]+"."+user["aoi"]+" "
cmdline+="|"+" "
cmdline+="psql -d "+user["dbname"]+" -h "+user["host"]+" -U "+user["user"]

## Create temp bash file for shp2pgsql
outputcsv=tempfile.gettempdir()+"\\tmp_bash.bat" # Define the csv output file name
f = open(outputcsv, 'w')
f.write(cmdline)
f.close()
print cmdline

set PGPASSWORD=tais
shp2pgsql -s 3857 -d -I F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\AOI_idjwi_2016.shp belgianmapath2016.aoi | psql -d Mapathon -h localhost -U tais


In [185]:
%%cmd
%Temp%\tmp_bash.bat

Microsoft Windows [version 6.1.7601]
Copyright (c) 2009 Microsoft Corporation. Tous droits r�serv�s.

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>%Temp%\tmp_bash.bat

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>set PGPASSWORD=tais 

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>shp2pgsql -s 3857 -d -I F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\AOI_idjwi_2016.shp belgianmapath2016.aoi   | psql -d Mapathon -h localhost -U tais 
SET
SET
               dropgeometrycolumn                
-------------------------------------------------
 belgianmapath2016.aoi.geom effectively removed.
(1 ligne)

DROP TABLE
BEGIN
CREATE TABLE
ALTER TABLE
                       addgeometrycolumn                        
----------------------------------------------------------------
 belgianmapath2016.aoi.geom SRID:3857 TYPE:MULTIPOLYGON DIMS:2 
(1 ligne)

INSERT 0 1
CREATE INDEX
COMMIT
ANALYZE

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>

Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]


#### Update the PostGis tables nearly creadted to ensure all geometries are valid

In [186]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Query
query="UPDATE "+user["schema"]+"."+user["aoi"]+" \
SET geom = ST_Multi(ST_CollectionExtract(ST_MakeValid(geom), 3)) \
WHERE ST_IsValid(geom) is not True"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
    
# Close cursor and communication with the database
cur.close()
db.close()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

## Create the bounding box (bbox) of the AOI

In [187]:
# Connect to the database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [188]:
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["bbox"])
# Make the changes to the database persistent
db.commit()

In [189]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Subquery1 
subquery1="SELECT ST_Envelope(geom) as the_geom FROM "+user["schema"]+"."+user["aoi"]

# Save the previous subquery results in a new table
query="CREATE TABLE "+user["schema"]+"."+user["bbox"]+" AS ("
query+=subquery1+")"

# Execute the query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
# Close cursor and communication with the database
cur.close()
db.close()

## Get upper, lower, right and left coordinates of the bbox (in WGS84).

In [190]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

In [191]:
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["bbox_coord"])
# Make the changes to the database persistent
db.commit()

In [192]:
### Save the previous subquery1 results in a new table
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

subquery1="WITH \
wgs84geom AS(\
SELECT  ST_Transform(the_geom, 4326) AS the_geom \
FROM "+user["schema"]+"."+user["bbox"]+") \
\
SELECT ST_Xmin(the_geom) AS west, ST_Xmax(the_geom) AS east, \
ST_Ymin(the_geom) AS south, ST_Ymax(the_geom) AS north \
FROM wgs84geom"

# Query
query="CREATE TABLE "+user["schema"]+"."+user["bbox_coord"]+" AS ("
query+=subquery1+")"

# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()
# Close cursor and communication with the database
cur.close()
db.close()

In [193]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT * FROM "+user["schema"]+"."+user["bbox_coord"]+" \
LIMIT "+str(nbrow)
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,west,east,south,north
0,28.953869,29.139637,-2.288842,-1.873235


In [194]:
## Save coordinates in variables
north_coord=round(df_bbox_coord.loc[0,['north']],6)
south_coord=round(df_bbox_coord.loc[0,['south']],6)
west_coord=round(df_bbox_coord.loc[0,['west']],6)
east_coord=round(df_bbox_coord.loc[0,['east']],6)

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Download OpenStreetMap data using Xapi

Here after, the OpenStreetMap (OSM) data will be downloaded for the AOI. For this prupose, OSM Xapi is used. Please read [this page](http://wiki.openstreetmap.org/wiki/Xapi) or [this one](http://wiki.openstreetmap.org/wiki/Overpass_API/XAPI_Compatibility_Layer) for extra information on Xapi. Please notice that the '@meta' is important to download informations about the osm features (last user, timestamp). Osm2pgsql use a .style file which manage how to osm data will be uploaded in the Postgis database. Please use [this .style file](https://github.com/tgrippa/Mapathon_HOT_OSM_WhatWeMapped/blob/master/default_OSMmetadata.style) wich allow to upload the metadata correctly.

In [195]:
import urllib

## Create the Xapi URL to download the data
print "OSM data will be downloaded for bbox (W:"+str(west_coord)+" S:"+str(south_coord)+" E:"+str(east_coord)+" N:"+str(north_coord)+")"

osm_api_base_url="http://www.overpass-api.de/api/xapi?*"
osm_api_url=osm_api_base_url+"[bbox="+str(west_coord)+","+str(south_coord)+","+str(east_coord)+","+str(north_coord)+"][@meta]"

osm_file=user["osmfolder"]+"/"+user["prefixosm"]+".osm"

## Retrieve the .osm file with the URL
urllib.urlretrieve (osm_api_url, osm_file)
print osm_api_url

OSM data will be downloaded for bbox (W:28.953869 S:-2.288842 E:29.139637N:-1.873235)
http://www.overpass-api.de/api/xapi?*[bbox=28.953869,-2.288842,29.139637,-1.873235][@meta]


# Import OSM layers in PostGis database

In [196]:
## Makes a 5sec stop in the process
import time
time.sleep(5)

## Build the osm2pgsql command-line
cmdline=""
cmdline="set PGPASSWORD="+user["password"]+"\n"
cmdline+="osm2pgsql -c -d "+user["dbname"]+" " 
cmdline+="-U "+user["user"]+" "
cmdline+="-H "+user["host"]+" "
cmdline+="--extra-attributes"+" "
cmdline+="-p "+user["prefixosm"]+" "
cmdline+="-S "+user["stylefile"]+" "
cmdline+=user["osmfolder"]+"/"+user["prefixosm"]+".osm"

## Create temp bash file for osm2pgsql
outputcsv=tempfile.gettempdir()+"\\tmp_bash.bat" # Define the csv output file name
f = open(outputcsv, 'w')
f.write(cmdline)
f.close()

In [197]:
%%cmd
%Temp%\tmp_bash.bat

Microsoft Windows [version 6.1.7601]
Copyright (c) 2009 Microsoft Corporation. Tous droits r�serv�s.

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>%Temp%\tmp_bash.bat

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>set PGPASSWORD=tais 

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>osm2pgsql -c -d Mapathon -U tais -H localhost --extra-attributes -p osm -S C:\Users\Admin_ULB\osm2pgsql-bin\default_TG.style F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016/osm.osm 

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>

osm2pgsql version 0.93.0-dev (64 bit id space)

Using built-in tag processing pipeline
Using projection SRS 3857 (Spherical Mercator)
Setting up table: osm_point
Setting up table: osm_line
Setting up table: osm_polygon
Setting up table: osm_roads
Allocating memory for sparse node cache
Node-cache: cache=800MB, maxblocks=12800*65536, allocation method=1

Reading in file: F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016/osm.osm
Using XML parser.
  parse time: 0s
Node stats: total(179354), max(4738114840) in 0s
Way stats: total(30788), max(460859657) in 0s
Relation stats: total(24), max(6143205) in 0s
Committing transaction for osm_point
Committing transaction for osm_line
Committing transaction for osm_polygon
Committing transaction for osm_roads
Using built-in tag processing pipeline
Using built-in tag processing pipeline
Using built-in tag processing pipeline
Using built-in tag processing pipeline

Going over pending ways...
	29804 ways are pending

Using 4 helpe

In [198]:
## Erase the temporary 'reclass_rule.csv' file
os.remove(outputcsv)

Here after, the osm table "line" and "polygon" will be moved to the current user's schema.

As HOT's tasks (Humanitarian OpenStreetMap Team) generally focus on highways, buildings and residential area, these features are the only one kept for the rest of this notebook. If you need something else, you should adapt the code.

In [199]:
## Move tables "lines" and "polygons" in current schema

# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["prefixosm"]+"_"+"line")
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon")
# Make the changes to the database persistent
db.commit()

# Copy table from public schema to current schema (only intersections with AOI):
subquery1="WITH \
lines AS(\
SELECT  (ST_Dump(ST_Intersection(l.way, aoi.geom))).geom AS the_geom, l.*  \
FROM "+user["schema"]+"."+user["aoi"]+" AS aoi, public."+user["prefixosm"]+"_"+"line"+" AS l \
WHERE l.highway is not null) \
\
SELECT * FROM lines"
# Query
query="CREATE TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"line AS ("
query+=subquery1+")"
# Execute the CREATE TABLE query 
cur.execute(query)

# Copy table from public schema to current schema (only intersections with AOI):
subquery1="WITH \
poly AS(\
SELECT  (ST_Dump(ST_Intersection(p.way, aoi.geom))).geom AS the_geom, p.*  \
FROM "+user["schema"]+"."+user["aoi"]+" AS aoi, public."+user["prefixosm"]+"_"+"polygon"+" AS p \
WHERE  p.building is not NULL OR p.landuse = 'residential' ) \
\
SELECT * FROM poly"
# Query
query="CREATE TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon AS ("
query+=subquery1+")"
# Execute the CREATE TABLE query 
cur.execute(query)
# Make the changes to the database persistent
db.commit()

# Drop 'way' column if exists:
cur.execute("ALTER TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"line DROP COLUMN IF EXISTS way")
# Drop table if exists:
cur.execute("ALTER TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon DROP COLUMN IF EXISTS way")
# Make the changes to the database persistent
db.commit()

Here after, the table in the "public" schema will be droped.

In [200]:
## Drop tables of public schema which will not be used anymore

# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS public."+user["prefixosm"]+"_"+"point")
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS public."+user["prefixosm"]+"_"+"roads")
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS public."+user["prefixosm"]+"_"+"line")
# Drop table if exists:
cur.execute("DROP TABLE IF EXISTS public."+user["prefixosm"]+"_"+"polygon")

# Make the changes to the database persistent
db.commit()

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Timestamp for Qgis timemanager plugin

## Add new colum 'forever' for Qgis timemanager

In [201]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Query:
cur.execute("ALTER TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"line ADD COLUMN IF NOT EXISTS forever timestamp(0) with time zone")
cur.execute("ALTER TABLE "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon ADD COLUMN IF NOT EXISTS forever timestamp(0) with time zone")

# Make the changes to the database persistent
db.commit()

In [206]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Open a cursor to perform database operations
cur=db.cursor()

# Query:
cur.execute("UPDATE "+user["schema"]+"."+user["prefixosm"]+"_"+"line SET forever=TIMESTAMP WITH TIME ZONE '2030-01-01 00:00:00+01'")
cur.execute("UPDATE "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon SET forever=TIMESTAMP WITH TIME ZONE '2030-01-01 00:00:00+01'")
# Make the changes to the database persistent
db.commit()

# Export shapefile

Here after, the tables 'line' and 'polygon' are exported from Postgis as shapefiles for gentle use in Qgis.

### Export lines table

In [207]:
## Build the pgsql2shp command-line
cmdline="set PGPASSWORD="+user["password"]+"\n"
cmdline+="pgsql2shp -f "
cmdline+=user["output_lineshp"]+" "
cmdline+=" -h "+user["host"]+" -u "+user["user"]+" "
cmdline+=user["dbname"]+" "
cmdline+=user["schema"]+"."+user["prefixosm"]+"_"+"line"

## Create temp bash file for pgsql2shp
outputcsv=tempfile.gettempdir()+"\\tmp_bash.bat" # Define the csv output file name
f = open(outputcsv, 'w')
f.write(cmdline)
f.close()

print cmdline

set PGPASSWORD=tais
pgsql2shp -f F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\output_line.shp  -h localhost -u tais Mapathon belgianmapath2016.osm_line


In [208]:
%%cmd
%Temp%\tmp_bash.bat

Microsoft Windows [version 6.1.7601]
Copyright (c) 2009 Microsoft Corporation. Tous droits r�serv�s.

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>%Temp%\tmp_bash.bat

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>set PGPASSWORD=tais 

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>pgsql2shp -f F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\output_line.shp  -h localhost -u tais Mapathon belgianmapath2016.osm_line 
Initializing... 
Done (postgis major version: 2).
Output shape: PolyLine
Dumping: XXXXXXXX [723 rows].

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>

In [209]:
## Erase the temporary 'reclass_rule.csv' file
os.remove(outputcsv)

### Export polygons table

In [210]:
## Build the pgsql2shp command-line
cmdline="set PGPASSWORD="+user["password"]+"\n"
cmdline+="pgsql2shp -f "
cmdline+=user["output_polygonshp"]+" "
cmdline+=" -h "+user["host"]+" -u "+user["user"]+" "
cmdline+=user["dbname"]+" "
cmdline+=user["schema"]+"."+user["prefixosm"]+"_"+"polygon"

## Create temp bash file for osm2pgsql
outputcsv=tempfile.gettempdir()+"\\tmp_bash.bat" # Define the csv output file name
f = open(outputcsv, 'w')
f.write(cmdline)
f.close()

print cmdline

set PGPASSWORD=tais
pgsql2shp -f F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\output_polygon.shp  -h localhost -u tais Mapathon belgianmapath2016.osm_polygon


In [211]:
%%cmd
%Temp%\tmp_bash.bat

Microsoft Windows [version 6.1.7601]
Copyright (c) 2009 Microsoft Corporation. Tous droits r�serv�s.

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>%Temp%\tmp_bash.bat

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>set PGPASSWORD=tais 

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>pgsql2shp -f F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016\output_polygon.shp  -h localhost -u tais Mapathon belgianmapath2016.osm_polygon 
Initializing... 
Done (postgis major version: 2).
Output shape: Polygon
Dumping: XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX [29024 rows].

F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager>

In [212]:
## Erase the temporary 'reclass_rule.csv' file
os.remove(outputcsv)

# Mapathon statistics

Here after, some basic stats are computed to inform on key infos of the mapathon. You could add extra stats if you want.

In [213]:
# Enter starting time of the mapathon (YYYY-MM-DD HH:MM:SS)
begintimemapping='2016-04-16 08:00:00'
# Enter ending time of the mapathon (YYYY-MM-DD HH:MM:SS)
endingtimemapping='2016-04-16 20:00:00'

### Number of unique mapper

In [214]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="WITH distinctuser_line AS(\
SELECT DISTINCT osm_user FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"line \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"'), \
\
distinctuser_polygon AS(\
SELECT DISTINCT osm_user FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"'), \
\
global_distinctuser AS(\
SELECT * FROM distinctuser_line \
UNION \
SELECT * FROM distinctuser_polygon)\
\
SELECT count(*) FROM global_distinctuser"

# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,count
0,177


### Number of editions by mapper

In [215]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="WITH \
all_edits AS(\
SELECT * FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"line \
UNION \
SELECT * FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon) \
\
SELECT osm_user, count(*) FROM all_edits \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
GROUP BY osm_user \
ORDER BY count(*) DESC"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,osm_user,count
0,GeoFrizz,422
1,lbataille,401
2,ouaip123,350
3,L'imaginaire,348
4,Anakil,343
5,MerkatorLeuven,335
6,ArticCynda,321
7,mclaeysb,317
8,lvdorpe,316
9,SoMeurrens,307


### Number of buildings mapped

In [216]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT count(*) FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND building is not null"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,count
0,23036


### Number of residential area mapped

In [272]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT count(*) FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"polygon \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND landuse='residential'"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,count
0,3


### Number of roads mapped

In [217]:
# Connect to an existing database
db=pg.connect(dbname=user["dbname"], user=user["user"], password=user["password"], host=user["host"])
# Number of line to show (please limit to 100 for saving computing time)
nbrow=500
# Query
query="SELECT count(*) FROM "+user["schema"]+"."+user["prefixosm"]+"_"+"line \
WHERE osm_timestamp >= '"+str(begintimemapping)+"' AND osm_timestamp <= '"+str(endingtimemapping)+"' \
AND highway is not null"
# Execute query through panda
df_bbox_coord=pd.read_sql(query, db)
# Show dataframe
df_bbox_coord.head(50)

,count
0,377


# Create map frames using Time Manager Plugin

**This step should be done outsite from this notebook.**

Use the [Qgis time manager plugin](https://github.com/anitagraser/TimeManager) to produced maps frames. More info could be found on the [Anita Graser's blog](https://anitagraser.com/2011/11/20/nice-animations-with-time-managers-offset-feature/). 

In the following cell of this notebook, please enter the path to the folder containing the frame for .gif and .mp4 creation purpose. More infos for this step can be found [here](http://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python).

In [268]:
## Enter the path to the folder with maps frames from TimeManager Plug-in
user["map_frames_folder"]='F:\Tais\Dropbox\BROL\Mapathon_Tais\Qgis_timemanager\Belgianmapathon2016/frames'

## Create GIF

For creation of .gif output, please first install [imageio](https://github.com/imageio/imageio).

In [293]:
import imageio
import glob
frames = []
framepattern=user["map_frames_folder"]+"//*.png"
frames=glob.glob(framepattern)

In [294]:
from PIL import Image
from numpy import array
imagestack = []
for frame in frames:
    img = Image.open(frame)
    arr = array(img)
    imagestack.append(arr)
imageio.mimsave(user["map_frames_folder"]+'/Mapathon2016accomplishment.gif', imagestack, fps=4)

## Create a movie

For creation of .mp4 output, please first install [imageio](https://github.com/imageio/imageio).

In [295]:
import imageio
import glob
frames = []
framepattern=user["map_frames_folder"]+"//*.png"
frames=glob.glob(framepattern)

In [296]:
## Download FFmpeg
imageio.plugins.ffmpeg.download()

In [297]:
from PIL import Image
from numpy import array
with imageio.get_writer(user["map_frames_folder"]+"//Mapathon2016accomplishment.mp4", mode='I', fps=4) as writer:
    for frame in frames:
        img = Image.open(frame)
        arr = array(img)
        writer.append_data(arr)